<a href="https://colab.research.google.com/github/abdulhafizsuherman/Python_GMB-Tea-Leaves/blob/master/projectGMB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 
drive.mount('/content/drive/') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
cd /content/drive/My Drive/Kuliah/Semester 8/TA

/content/drive/My Drive/Kuliah/Semester 8/TA


In [ ]:
import itertools
import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from imutils import paths
from keras.layers import AveragePooling2D
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
# TODO grab all image paths
print("[INFO] loading images...")
imagePaths = paths.list_images("proses2_datasetbaru(channelGreen)")
data = []
labels = []

[INFO] loading images...


In [ ]:
for imagePath in imagePaths:
    image = Image.open(imagePath)
    image = np.array(image.resize((192, 192))) / 255.0
    image = np.expand_dims(image, axis=-1)  # penambahan dimensi array untuk proses 1 kanal
    data.append(image)

    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

# TODO encode the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

print(lb)
print(labels)

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)
[[0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


In [ ]:
# TODO perform a training and testing split
''' test images : testX
    test labels : testY
    train images : trainX
    train labels : trainY'''

(trainX, testX, trainY, testY) = train_test_split(np.array(data),
                                                  np.array(labels), test_size=0.25)
print("[INFO] Shape train and test array...")
print("ukuran train X: ", trainX.shape)
print("ukuran test X: ", testX.shape)
print("ukuran train Y: ", trainY.shape)
print("ukuran test Y: ", testY.shape)

In [ ]:
# TODO Initialising the CNN with Architecture LeNet-5

class Lenet5:
    def __init__(self, imgRows, imgCols, numChannels):
        self.numChannels = numChannels
        self.imgRows = imgRows
        self.imgCols = imgCols

    def architecture(self):
        # initialize the model
        classifier = Sequential()
        inputShape = (self.imgRows, self.imgCols, self.numChannels)

        kernel_size = (5, 5)
        strides = (2, 2)
        pool_size = (2, 2)
        # Step 1 - Convolution(C1)
        classifier.add(Conv2D(6, kernel_size=kernel_size, strides=strides,
                              input_shape=inputShape, activation='relu'))

        # Step 2 - Average Pooling(S2)
        classifier.add(AveragePooling2D(pool_size=pool_size, strides=strides))

        # Step 3 - Convolution(C3)
        classifier.add(Conv2D(16, kernel_size=kernel_size, activation='relu'))

        # step 4 - Average Pooling(S4)
        classifier.add(AveragePooling2D(pool_size=strides, strides=strides))

        # Step 5 - Flattening
        classifier.add(Flatten())

        # Step 4 - Full connection
        classifier.add(Dense(units=120, activation='relu'))
        classifier.add(Dense(units=84, activation='relu'))
        classifier.add(Dense(units=11, activation='softmax'))
        return classifier


model = Lenet5(192, 192, 1).architecture()  # define method
# TODO train the model using the Adam optimizer / compiling CNN

print("[INFO] training network...")
opt = Adam(lr=0.001, decay=1e-2 / 50)  
# opt = Adam(lr=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

""" Since we are using the categorical cross-entropy loss function, we need to apply the to_categorical  function 
which converts our labels from integers to a vector, where each vector ranges from [0, classes] . This function 
generates a vector for each class label, where the index of the correct label is set to 1 and all other entries are 
set to 0. """

history = model.fit(trainX, trainY, validation_data=(testX, testY), epochs=100, batch_size=32)

# ======================================================================================================================

# TODO evaluate the network

print("[INFO] evaluating network using classification report...")

y_pred = model.predict(testX, batch_size=32)
testY_argm = testY.argmax(axis=1)
y_pred1_argm = y_pred.argmax(axis=1)

print('Accuracy Score :', accuracy_score(testY_argm, y_pred1_argm))

clas_report = classification_report(testY_argm, y_pred1_argm, target_names=lb.classes_)
print("==Classification Report==")
print(clas_report)


# ======================================================================================================================

# TODO confusion matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap='Blues'):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


# Compute confusion matrix

cnf_matrix = confusion_matrix(testY_argm, y_pred1_argm)
np.set_printoptions(precision=2)

print("==Confusion Matrix==")
print(cnf_matrix)

# Plot non/normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=lb.classes_, normalize=True)
plt.show()

plt.figure()
plot_confusion_matrix(cnf_matrix, classes=lb.classes_, normalize=False)
plt.show()

# TODO plotting Value accuracy

# Plot training & validation accuracy values
plt.figure()
plt.style.use('ggplot')
plt.plot(history.history['acc'], 'b-')
plt.plot(history.history['val_acc'], 'r-')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train Images', 'Test Images'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.figure()
plt.style.use('ggplot')
plt.plot(history.history['loss'], 'b-o')
plt.plot(history.history['val_loss'], 'r-v')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train Images', 'Test Images'], loc='upper left')
plt.show()